In [1]:
from telethon import TelegramClient, events
from pymongo import MongoClient
import datetime
import pandas as pd
import yaml
import os
from multiprocessing import Process

In [2]:
channels = pd.read_csv('telegram_channels.csv') #load a list of channels you want to stream
channels = list(channels['Channels'])

In [3]:
if os.path.isfile("config.yml"):                #load the config file
    with open("config.yml", 'r') as ymlfile:
        config_file = yaml.load(ymlfile)

D:\Anaconda3\envs\d\lib\site-packages\ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
api_id = config_file.get('API_ID')     #the api id and hash are contained in the config file
api_hash = config_file.get('API_HASH')

In [5]:
mongo_client = MongoClient(config_file.get('MONGO_CONNECTION', 'mongodb://localhost/')) #conect to MongoDb
db = mongo_client[config_file.get('MONGO_DB')]
collection = db.my_collection                          #create a variable for your collection

In [6]:
client = TelegramClient('session_name', api_id, api_hash) #connect to telegram

In [7]:
def get_messages(channel):                               #method to stream messages of the channel
    @client.on(events.NewMessage(chats=(channel)))
    async def my_event_handler(event):
        print(event.message.message)                        # to check if the message is recived
        collection.insert_one({'channel':channel, 'id':event.message.id, 'text':event.message.message})   # inserting into mongodb

In [8]:
for channel in channels:               #call the message getter function
    get_messages(channel)

In [9]:

    await client.start()                       #start the telegram clinent
    await client.run_until_disconnected()

Hi
I am sending message to my self


CancelledError: 